In [1]:
%pip install torch -q 
%pip install transformers -q 
%pip install numpy -q 
%pip install langchain -q 
%pip install langchain_community -q 
%pip install langchain-chroma -q 
%pip install langchain_transformers -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Initialize HuggingFace LLM


In [2]:
# Install python-dotenv to load .env files
%pip install python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from langchain_community.llms import HuggingFaceHub

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv('HUGGINGFACE_API_TOKEN')

if not api_key:
    raise ValueError("HUGGINGFACE_API_TOKEN not found in .env file. Please check your .env file.")

# Initialize the HuggingFace llm
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    model_kwargs={"temperature": 0.1, "max_length": 500},
    huggingfacehub_api_token=api_key
)

print("✅ HuggingFace LLM initialized successfully!")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_43356\2339488867.py:15: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


✅ HuggingFace LLM initialized successfully!


Initialize Embedding model

In [5]:
%pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install hf_xet

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 3.4 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.8 MB 2.6 MB/s eta 0:00:01
   ---------------------- ----------------- 1.6/2.8 MB 2.3 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embedding_model=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

Initialize output parser

In [11]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

Load the Pdf Document

In [12]:
%pip install pypdf -qU

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_community.document_loaders import PyPDFLoader

#Load the PDF document
loader=PyPDFLoader("Chapter 01.pdf")

docs=loader.load()

In [16]:
docs[0]

Document(metadata={'producer': 'Microsoft® PowerPoint® LTSC', 'creator': 'Microsoft® PowerPoint® LTSC', 'creationdate': '2024-10-12T10:35:42+05:30', 'title': '', 'author': 'Roshani Wijesuriya', 'moddate': '2024-10-12T10:35:42+05:30', 'source': 'Chapter 01.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='IN2311:Operating systems\nRoshani Wijesuriya')

Split Document into Chunks

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Initialize the text splitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)

#Split the document in to chunks
splits=text_splitter.split_documents(docs)

In [19]:
len(splits)

23

Create Vector Store and Retriver

In [21]:
from langchain_chroma import Chroma

#Create vector store from the document chunks
vectorstore=Chroma.from_documents(documents=splits,embedding=embedding_model)

In [22]:
#Create a retriver from the vector store
retriever=vectorstore.as_retriever()